# **FOUNDATION OF COMPUTER SCIENCE PROJECT** 

### University of Milan - Bicocca
### Data Science
Giovanna Saracino matricola **841799**

In [1]:
import pandas as pd
import numpy as np
import datetime
import random

In [2]:
#import datasets
#/content/drive/My Drive/Computer Science/
loans_lenders = pd.read_csv('loans_lenders.csv')
loans = pd.read_csv('loans.csv')
lenders = pd.read_csv('lenders.csv')
country_stats = pd.read_csv('country_stats.csv')

### 1) **Normalize** the loans_lenders table. In the normalized table, each row must have one loan_id and one lender.

Controllo il dataset *loans_lenders*, limitando l'output alle prime 5 righe.

In [3]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


Si nota che per ogni loan_id ci sono più lenders.

Per avere una tabella in cui ogni riga ha un loan_id e un lender, ho bisogno di **dividere l'attributo *lenders*** per avere una lista di lenders e successivamente attraverso la funzione explode() **duplicare gli indici** (loan_id) per avere un loan_id e un lender.

In [4]:
loans_lenders_normalized = (loans_lenders.set_index(['loan_id'])      #set loan_id as dataframe index
                   .apply(lambda x: x.str.split(', ').explode())    #split each lender and then transform each element of a list-like to a row, replicating index values
                   .reset_index())                                

loans_lenders_normalized

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


### 2) For each loan, **add a column duration** corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

Controllo il dataset *loans*.

Controllo le variabili *disburse_time* e *planned_expiration_time*.

In [5]:
loans.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model'],
      dtype='object')

In [6]:
loans[['planned_expiration_time', 'disburse_time']].head()

,planned_expiration_time,disburse_time
0,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000
1,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000
2,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000
3,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000
4,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000


Per ottenere il numero di giorni tra il *disburse time* e *planned expiration time*, bisogna convertire prima questi attributi in **datetime** e considerare solo il campo data. 

Successivamente, si applica la **differenza** per trovare il numero di giorni.

In [7]:
def findDate(df, attr):
    return pd.to_datetime(df[attr]).dt.date

In [8]:
loans["duration"] =  findDate(loans, "planned_expiration_time") - findDate(loans, "disburse_time")

#view the head of the table
loans[["loan_id", "duration"]]

,loan_id,duration
0,657307,54 days
1,657259,96 days
2,658010,37 days
3,659347,35 days
4,656933,58 days
...,...,...
1419602,988180,40 days
1419603,988213,39 days
1419604,989109,51 days
1419605,989143,63 days


Verifico che, se una delle due date è mancante, anche l'attributo **duration** deve essere *missing*.

In [9]:
loans[(loans["disburse_time"].isnull()) | (loans["planned_expiration_time"].isnull())][['planned_expiration_time', 'disburse_time','duration']].head() #374645 rows

,planned_expiration_time,disburse_time,duration
64,NaN,2010-11-18 08:00:00.000 +0000,NaT
65,NaN,2010-11-25 08:00:00.000 +0000,NaT
66,NaN,2010-11-25 08:00:00.000 +0000,NaT
67,NaN,2010-11-16 08:00:00.000 +0000,NaT
69,NaN,2010-11-10 08:00:00.000 +0000,NaT


### 3) Find the **lenders** that have **funded at least twice**.

Nel dataframe loans, è presente lo *status* di ogni loan. Si seleziona solo lo status *funded*.

In [10]:
loans.status.value_counts()

funded         1350340
expired          59081
refunded          6578
fundRaising       3608
Name: status, dtype: int64

Per ottenere i lenders che hanno investito almeno due volte, bisogna effettuare un merge tra i df *loans* e *loans_lenders_normalized*. 

Sul df *loans* trovo tutti i loan_id il cui stato è **funded**, e successivamente seleziono i lender che hanno effettuato un prestito **almeno due volte**.

In [11]:
loans_funded = pd.merge(loans['loan_id'].loc[loans['status'] == 'funded'], loans_lenders_normalized,
                        on = 'loan_id') 
loans_funded.head()

,loan_id,lenders
0,657307,spencer5657
1,657307,matthew8640
2,657307,larry71496105
3,657259,ltr
4,657259,andrew5306


In [12]:
loans_funded['lenders'].value_counts().loc[lambda x: x>=2] 

gooddogg1           167512
trolltech4460       141268
gmct                123164
nms                  96713
themissionbeltco     74067
                     ...  
luqq                     2
leslie65436147           2
scott6541                2
trevor7891               2
robert8294               2
Name: lenders, Length: 893981, dtype: int64

### 4) For each country, compute how many loans have involved that country as **borrowers**.

Per calcolare quanti prestiti hanno coinvolto un determinato Paese, si calcolano le occorrenze di ogni Paese nel dataframe *loans*.

In [13]:
print("Number of rows in loans: ",loans.shape[0])
print("Unique values of loan_id: ", len(loans['loan_id'].unique()))

Number of rows in loans:  1419607
Unique values of loan_id:  1419607


In [14]:
loans['country_name'].value_counts()
#1 loans per row

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Papua New Guinea         1
Botswana                 1
Canada                   1
Uruguay                  1
Mauritania               1
Name: country_name, Length: 96, dtype: int64

### 5) For each country, compute the **overall amount of money borrowed**.

Per calcolare l'importo complessivo del denaro *preso in prestito* per ogni Paese, **raggruppo per *country_name*** e **sommo l'importo del prestito**.

In [15]:
country_amount = loans.groupby('country_name')['loan_amount'].sum()#.reset_index()

country_amount

country_name
Afghanistan        1967950.0
Albania            4307350.0
Armenia           22950475.0
Azerbaijan        14784625.0
Belize              150175.0
                     ...    
Vietnam           24681100.0
Virgin Islands       10000.0
Yemen              3444000.0
Zambia             1978975.0
Zimbabwe           5851875.0
Name: loan_amount, Length: 96, dtype: float64

### 6) Like the previous point, but expressed as a **percentage of the overall amount lent**.

A differenza del punto precedente, considero gli importi che sono stati *prestati*. Quindi, seleziono l'attributo *funded_amount* che è l'importo finanziato. Infine, calcolo la **percentuale** di denaro prestato sul denaro preso in prestito.

In [16]:
country_lent = loans.groupby('country_name')['funded_amount'].sum()

country_lent / country_amount

country_name
Afghanistan       0.998323
Albania           0.926898
Armenia           0.883846
Azerbaijan        0.959872
Belize            1.000000
                    ...   
Vietnam           0.950680
Virgin Islands    0.000000
Yemen             0.969810
Zambia            0.966788
Zimbabwe          0.994305
Length: 96, dtype: float64

### 7) Like the three previous points, but **split for each year** (with respect to disburse time).



*   Per ogni *country_name* e *year* di erogazione, calcolo quanti loan hanno coinvolto un determinato Paese come "debitore"

Poichè nel df loan ogni riga corrisponde ad un loan diverso, utilizzo la funzione **size** che restituisce il numero di righe, cioè il numero di loan per *country_name* e *disburse_year*


In [17]:
print( "Unique loan_id: ", len(loans['loan_id'].unique()))
print("Number of rows of loans dataframe: ", loans.shape[0])

Unique loan_id:  1419607
Number of rows of loans dataframe:  1419607


In [18]:
loans["disburse_year"]= pd.to_datetime(loans["disburse_time"]).dt.year

In [19]:
loans.groupby(['country_name', 'disburse_year']).size() 

country_name  disburse_year
Afghanistan   2007.0            408
              2008.0            370
              2009.0            678
              2010.0            632
              2011.0            247
                               ... 
Zimbabwe      2013.0            426
              2014.0           2078
              2015.0            600
              2016.0            808
              2017.0           1079
Length: 748, dtype: int64

*  Per ogni Paese, calcolo l'**importo totale** di denaro preso in prestito **per anno**. 

In [20]:
loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum()

country_name  disburse_year
Afghanistan   2007.0            194975.0
              2008.0            365375.0
              2009.0            585125.0
              2010.0            563350.0
              2011.0            245125.0
                                 ...    
Zimbabwe      2013.0            678525.0
              2014.0           1311575.0
              2015.0            723625.0
              2016.0            788600.0
              2017.0           1237600.0
Name: loan_amount, Length: 748, dtype: float64

*  Come i punti precedenti, ma espresso come **percentuale** di denaro totale preso in prestito.


Dunque, divido l'ammontare del denaro **prestato** sull'ammontare del denaro **preso in prestito** e lo esprimo in percentuale

In [21]:
(loans.groupby(['country_name', 'disburse_year'])['funded_amount'].sum() / loans['loan_amount'].sum()) * 100

country_name  disburse_year
Afghanistan   2007.0           0.016503
              2008.0           0.030926
              2009.0           0.049247
              2010.0           0.047683
              2011.0           0.020748
                                 ...   
Zimbabwe      2013.0           0.057432
              2014.0           0.110710
              2015.0           0.061209
              2016.0           0.066660
              2017.0           0.102767
Name: funded_amount, Length: 748, dtype: float64

### 8) For each lender, compute the **overall amount of money lent**. For each loan that has more than one lender, you must assume that all lenders contributed the same amount



1.   Unisco i df *loans* e *loans_lenders_normalizes*
2.   Calcolo il contributo di ogni membro per loan (assumendo che a ogni lender corrisponde la stessa somma)
3.  Infine, raggruppo per lender e sommo il denaro



In [22]:
lent_lenders = pd.merge(loans_lenders_normalized, loans[['loan_id', 'funded_amount', 'num_lenders_total']], 
                              on = 'loan_id')

In [23]:
lent_lenders.head(5)

,loan_id,lenders,funded_amount,num_lenders_total
0,483693,muc888,1225.0,44
1,483693,sam4326,1225.0,44
2,483693,camaran3922,1225.0,44
3,483693,lachheb1865,1225.0,44
4,483693,rebecca3499,1225.0,44


In [24]:
lent_lenders['lender_contribution'] = lent_lenders['funded_amount'] / lent_lenders['num_lenders_total']

In [25]:
lent_lenders.head()

,loan_id,lenders,funded_amount,num_lenders_total,lender_contribution
0,483693,muc888,1225.0,44,27.840909
1,483693,sam4326,1225.0,44,27.840909
2,483693,camaran3922,1225.0,44,27.840909
3,483693,lachheb1865,1225.0,44,27.840909
4,483693,rebecca3499,1225.0,44,27.840909


In [26]:
lent_lenders.groupby('lenders')['lender_contribution'].sum().reset_index(name = 'lent')

,lenders,lent
0,000,1512.095370
1,00000,1249.127691
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932
...,...,...
1383794,zzmcfate,57535.947017
1383795,zzpaghetti9994,44.247788
1383796,zzrvmf8538,480.818179
1383797,zzzsai,236.888048


### 9) For each country, compute the **difference between the overall amount of money lent and the overall amount of money borrowed**.

##### Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Controllo il dataset *lenders*

In [27]:
lenders['country_code'].isnull().value_counts()

True     1458635
False     890539
Name: country_code, dtype: int64

In [31]:
def found_distribution(country_code):
    random.seed(1234)
    results = list()
    #country_code unique values without na
    values = pd.unique(country_code.dropna())
    results.append(values)
    #prob computed as occurrences of country/number of country code
    probs = country_code.dropna().value_counts() / len(country_code.dropna())
    results.append(probs)
    #index of row where the country code is null
    index_rows_nan = country_code.index[country_code.isnull()]
    results.append(index_rows_nan)
    #n is the number of rows with nan
    n = len(index_rows_nan)
    results.append(n)
    return (results)

Attraverso la funzione *found_distribution()* trovo la distribuzione 

In [32]:
results_function = found_distribution(lenders['country_code'])

Per ottenere un random country_code con una distribuzione calcolata con found_distribution() utilizzo la funzione **random.choice()** che prende in input:

* array: un campione random viene generato dai suoi elementi  (country_code unique values without na)
* probabilities: le probabilità associate ad ogni elemento nell'array (prob computed as occurrences of country/number of country code)
* size: output shape (number of rows without na)
* index (index of row where the country code is null)

In [33]:
random_country_code = pd.Series(np.random.choice(results_function[0], p = results_function[1],
                                                size = results_function[3]), index = results_function[2])
random_country_code

0          US
1          US
2          US
3          US
4          PK
           ..
2349167    US
2349168    US
2349169    US
2349170    US
2349172    US
Length: 1458635, dtype: object

Ora inserisco i *country_code* presenti in random_country_code nel dataframe *lenders.country_code* dove prima vi erano gli NA.

In [34]:
lenders['country_code'].fillna(random_country_code, inplace = True)

In [35]:
lenders['country_code'].isna().value_counts()

False    2349174
Name: country_code, dtype: int64

Ora, per ogni country_code, calcolo l'ammontare dei soldi **presi in prestito**

In [36]:
borrowed_by_country = loans.groupby('country_code')['loan_amount'].sum().reset_index(name = 'borrowed')
borrowed_by_country

,country_code,borrowed
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0
...,...,...
90,XK,3083025.0
91,YE,3444000.0
92,ZA,1006525.0
93,ZM,1978975.0


Per ogni country_code calcolo l'ammontare totale di denaro **prestato**. 

Nel df *lent_lenders* creato nel punto 8), ho l'informazione riguardante il denaro prestato da ogni lender.

Dal df *lenders* seleziono solo le colonne riguardanti il nome e il country_code del lender.

In [37]:
print("Lenders columns are: ", lenders.columns)
print("***************************")
print("Lent lenders columns are: ", lent_lenders.columns)

Lenders columns are:  Index(['permanent_name', 'display_name', 'city', 'state', 'country_code',
       'member_since', 'occupation', 'loan_because', 'loan_purchase_num',
       'invited_by', 'num_invited'],
      dtype='object')
***************************
Lent lenders columns are:  Index(['loan_id', 'lenders', 'funded_amount', 'num_lenders_total',
       'lender_contribution'],
      dtype='object')


In [38]:
lent_lenders = pd.merge(lent_lenders, lenders[['permanent_name', 'country_code']], #lent_lenders contains funded_amount for each lenders
                             left_on = 'lenders', right_on = 'permanent_name')

In [39]:
pd.set_option('display.float_format', lambda x: '%.5f' % x) #format scientific notation

In [40]:
lent_per_country = lent_lenders.groupby('country_code')['lender_contribution'].sum().reset_index(name = 'lent')

Ora, unisco le informazioni riguardanti il denaro prestato e quello preso in prestito (borrowed). 

Utilizzo il df creato precedentemente *borrowed_by_country*.

In [41]:
lent_borrowed_by_country = pd.merge(lent_per_country, borrowed_by_country, on = 'country_code', 
                                    how = 'outer')

lent_borrowed_by_country.fillna(0, inplace = True)

lent_borrowed_by_country['difference'] = lent_borrowed_by_country['lent'] - lent_borrowed_by_country['borrowed']

lent_borrowed_by_country

,country_code,lent,borrowed,difference
0,AD,11491.70479,0.00000,11491.70479
1,AE,1689830.56678,0.00000,1689830.56678
2,AF,102534.94933,1967950.00000,-1865415.05067
3,AG,606.67996,0.00000,606.67996
4,AI,643.05934,0.00000,643.05934
...,...,...,...,...
229,YT,9554.90742,0.00000,9554.90742
230,ZA,487516.02248,1006525.00000,-519008.97752
231,ZM,33688.09131,1978975.00000,-1945286.90869
232,ZW,6662.86141,5851875.00000,-5845212.13859


### 10) Which country has the **highest ratio between the difference computed at the previous point and the population**.

Per avere informazioni riguardo al Paese con il più alto rapporto tra la differenza calcolata al punto precedente e la popolazione, visualizzo innanzitutto il dataframe *country_stats*.

In [50]:
country_stats.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.90000,0.62356,68.32200,11.69659,6.29883,5663.47480,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.00000,0.52710,53.05700,9.97048,6.00000,5442.90126,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.20000,0.76168,76.97200,13.29909,8.55498,16383.10668,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.50000,0.55035,66.36500,8.10691,5.08946,5031.17307,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.50000,0.57882,71.98500,10.17871,5.24158,3341.49072,Bangladesh


Unisco il dataset *lent_borrowed_by_country* e il dataset *country_stats* su **country_code**. 

Seleziono l'attributo 'population_below_poverty_line' che servirà nel punto 11).

In [42]:
lent_borrowed_by_country = pd.merge(lent_borrowed_by_country, country_stats[['country_code', 'population', 
                                    'population_below_poverty_line']], 
                                     on = 'country_code')

lent_borrowed_by_country = lent_borrowed_by_country.set_index('country_code')

#idxmax() return index of first occurrence of maximum
(lent_borrowed_by_country['difference'] / lent_borrowed_by_country['population']).idxmax()

'NO'

Il country_code 'NO' ha il rapporto più alto tra differenza e popolazione.

### 11) Which country has the **highest ratio between the difference computed at point 9 and the population that is not below the poverty line**.

Poiché la popolazione sotto la linea di povertà è espressa in percentuale, ottengo la percentuale di persone NON in povertà in questo modo:

In [43]:
lent_borrowed_by_country['population_not_below_poverty_line'] = 100 - lent_borrowed_by_country['population_below_poverty_line']

Successivamente, calcolo il numero di abitanti che non sono sotto la soglia di povertà

In [44]:
lent_borrowed_by_country['abs_population_not_below_poverty_line'] = (lent_borrowed_by_country['population'] * lent_borrowed_by_country['population_not_below_poverty_line']) / 100

In [45]:
 (lent_borrowed_by_country['difference'] / lent_borrowed_by_country['abs_population_not_below_poverty_line']).idxmax()

'CA'

Il paese "CA" ha il rapporto più alto tra la differenza e la popolazione che non è al di sotto della soglia di povertà.

### 12) For each year, compute the **total amount of loans**.


#### Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD 31 / (31+365+30) = 363.85 for 2016, 5000USD 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

Per ottenere il numero di giorni tra la data di inizio e la data di fine, utilizzo la funzione *day_calc()*.

*  Creo una lista con la prima data, il 1 Gennaio di ogni anno tra la data di inizio e la data di fine, e la data di fine.    
*  Creo una struttura chiave-valore in cui la chiave è l'anno e il valore corrisponde al numero di giorni tra la chiave corrente e la chiave successiva. 

**day_calc()** restituisce il totale dei giorni che intercorrono tra date1 e date2, dividendo per anno.

In [46]:
def day_calc(date1, date2):
    
    #control to avoid negative values
    (begin,end) = (date1,date2) if date1<date2 else (date2,date1) 
    
    #create a list where the first date is the start date, the second value is Jenuary first, and so on until the end date
    date_list = list() 
    
    date_list.append(begin)
    
    #for each year between begin date and end date append 1/01/year
    for y in range(begin.year+1, end.year+1): 
        date_list.append(pd.Timestamp(year=y, month=1, day=1, tz='utc')) 
        
    date_list.append(end)

    #create a dictionary
    res = dict()
    
    #the years become the key and the value is the difference (in days) between i+1 key and i key
    for i in range(len(date_list)-1):
        res[date_list[i].year] = (date_list[i+1] - date_list[i]).days 

    #create the series in which the index is the key
    return pd.Series(res, name='DateValue')

Converto gli attributi *planned_expiration_time* e *disburse_time* in datetime.

In [47]:
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])

In [48]:
#drop na values from loan
loans_drop = loans.dropna()
loans_drop['duration'] = loans_drop['duration'].dt.days
#for each loan in loans_drop 
days = loans_drop.apply(lambda x: day_calc(x['planned_expiration_time'],x['disburse_time']), axis = 1) 
days.head() #x is loan

C:\Users\giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,2011,2012,2013,2014,2015,2016,2017,2018
3,nan,nan,nan,34.00000,nan,nan,nan,nan
9,nan,nan,nan,68.00000,nan,nan,nan,nan
11,nan,nan,nan,36.00000,nan,nan,nan,nan
13,nan,nan,5.00000,84.00000,nan,nan,nan,nan
18,nan,nan,nan,nan,46.00000,nan,nan,nan


In [49]:
#fill na values with 0
days.fillna(0, inplace= True)
#concat loan_id, òpan_amount and duration
days = pd.concat([days, loans_drop['loan_id']
                 , loans_drop['loan_amount']
                 , loans_drop['duration'] ]
                , axis = 1, sort = True)

days.head()

,2011,2012,2013,2014,2015,2016,2017,2018,loan_id,loan_amount,duration
3,0.00000,0.00000,0.00000,34.00000,0.00000,0.00000,0.00000,0.00000,659347,625.00000,35
9,0.00000,0.00000,0.00000,68.00000,0.00000,0.00000,0.00000,0.00000,662117,1825.00000,68
11,0.00000,0.00000,0.00000,36.00000,0.00000,0.00000,0.00000,0.00000,659929,950.00000,36
13,0.00000,0.00000,5.00000,84.00000,0.00000,0.00000,0.00000,0.00000,661165,300.00000,90
18,0.00000,0.00000,0.00000,0.00000,46.00000,0.00000,0.00000,0.00000,956960,375.00000,46


Ora posso dividere [2011,...,2018] per durata e moltiplico per *loan_amount*

*  Esempio: nella tabella precedente abbiamo 5 giorni nel 2013 e 85 giorni (considerando il rounding) nel 2014 (index 13). Quindi, possiamo calcolare 5/90 x 300 = 16,6 -> questo è l'ammontare del 2013. Ora posso calcolare 85/90 x 300 = 283,4. La somma è 300

In [50]:
amounts = days[[2011,2012,2013,2014,2015,2016,2017,2018
                        ]].div(days.duration, axis=0).mul(days.loan_amount, axis = 0)
amounts.head()

,2011,2012,2013,2014,2015,2016,2017,2018
3,0.00000,0.00000,0.00000,607.14286,0.00000,0.00000,0.00000,0.00000
9,0.00000,0.00000,0.00000,1825.00000,0.00000,0.00000,0.00000,0.00000
11,0.00000,0.00000,0.00000,950.00000,0.00000,0.00000,0.00000,0.00000
13,0.00000,0.00000,16.66667,280.00000,0.00000,0.00000,0.00000,0.00000
18,0.00000,0.00000,0.00000,0.00000,375.00000,0.00000,0.00000,0.00000


In [51]:
amounts = pd.concat([days['loan_id'], amounts],  axis = 1 ) 

amounts = amounts.melt(id_vars=["loan_id"], 
        var_name="Year", 
        value_name="amount_per_year") # .melt() returns an unpivot transformation on df
amounts.head()

,loan_id,Year,amount_per_year
0,659347,2011,0.00000
1,662117,2011,0.00000
2,659929,2011,0.00000
3,661165,2011,0.00000
4,956960,2011,0.00000


In [52]:
amounts.groupby(['loan_id', 'Year']).sum()

amount_per_year
loan_id Year                 
372752  2011        180.00000
        2012        560.00000
        2013          0.00000
        2014          0.00000
        2015          0.00000
...                       ...
1443953 2014          0.00000
        2015          0.00000
        2016          0.00000
        2017        175.00000
        2018        700.00000

[3830696 rows x 1 columns]